# Analysis of hearing impairment phenotypes

This notebook applies the `Get_Job_Script.ipynb` to automatically generate the sbatch scripts to run in Yale's cluster. The end result is to apply [various LMM workflows](https://github.com/statgenetics/UKBB_GWAS_dev/tree/master/workflow) to perform association analysis in different hearing impairment traits, do clumping analysis and extract associated regions.

The phenotypes analyzed are:

1. Hearing aid f.3393
2. Hearing difficulty f.2247
3. Hearing difficulty with background noise f.2257
4. Combined phenotype f.2247 & f.2257

## File paths on Yale cluster

- Genotype files in PLINK format:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv`
- Genotype files in bgen format:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/`
- Summary stats for imputed variants BOLT-LMM:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data`
- Summary stats for inputed variants FastGWA:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data`
- Phenotype files:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment`
- Relationship file:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620`

## 08/31/20 analysis

On the cluster, open up this notebook using the JupyterLab server you set up via the ssh channel, then run the following cells,

## Bash variables for workflow configuration

In [3]:
# Common variables
tpl_file=../farnam.yml
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated082020removedwithdrawnindiv.bed
sampleFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample
bgenFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr{1..22}_v3.bgen`
unrelated_samples=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620
formatFile_fastgwa=~/project/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=~/project/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=~/project/UKBB_GWAS_dev/data/saige_template.yml
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
# Container
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif
# LMM directories
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/
lmm_dir_saige=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/SAIGE_results/results_imputed_data/
lmm_dir_regenie=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/REGENIE_results/results_imputed_data/
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_imp-fastgwa.sbatch
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_imp-bolt.sbatch
lmm_sbatch_saige=../output/$(date +"%Y-%m-%d")_imp-saige.sbatch
lmm_sbatch_regenie=../output/$(date +"%Y-%m-%d")_imp-regenie.sbatch
## LMM variables 
LDscoresFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/LDSCORE.1000G_EUR.tab.gz
geneticMapFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genetic_map_hg19_withX.txt.gz
covarMaxLevels=10
numThreads=20
bgenMinMAF=0.001
bgenMinINFO=0.8
lmm_job_size=1
ylim=0
### Specific to FastGWA
grmFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.grm.sp
### Specific to SAIGE
bgenMinMAC=4
trait_type=binary
loco=TRUE
sampleCol=IID
### Specific to REGENIE
bsize=1000
lowmem=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/REGENIE_results/results_imputed_data/
trait=bt
minMAC=5
reverse_log_p=True
# LD clumping directories
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/
clumping_sos=~/project/bioworkflows/GWAS/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_imp_ldclumping.sbatch
## LD clumping variables
# For sumtastsFiles if more than one provide each path
bfile_ref=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.1210.ref_geno.bed
# Changes dependending upon which traits are analyzed
# In this case tinnitus only
sumstatsFiles=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus/*.snp_stats.gz
ld_sample_size=1210
clump_field=P
clump_p1=5e-08
clump_p2=1
clump_r2=0.2
clump_kb=2000
clump_annotate=BP
numThreads=20
clump_job_size=1
clumpFile= 
clumregionFile=
# Region extraction directories
extract_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/tinnitus
extract_sos=~/project/bioworkflows/GWAS/Region_Extraction.ipynb
extract_sbatch=../output/$(date +"%Y-%m-%d")_tinnitus_imp-region.sbatch
## Region extraction variables
region_file=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/
geno_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/UKBB_bgenfilepath.txt
sumstats_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/
extract_job_size=10

## 1. Hearing aid user f.3393

### FastGWA job only white British

In [ ]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f3393_hearing_aid
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f3393_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_Hearing_aid_f3393
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_Hearing_aid_f3393
phenoCol=hearing_aid_cat
covarCol=sex
qCovarCol=age_final_aid

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

### FastGWA all whites

In [2]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f3393_hearing_aid_expandedwhite
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f3393_expandedwhite_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/120120_UKBB_Hearing_aid_f3393_expandedwhite
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/120120_UKBB_Hearing_aid_f3393_expandedwhite
phenoCol=hearing_aid_cat
covarCol=sex
qCovarCol=age_final_aid

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-12-02_f3393_expandedwhite_imp-fastgwa.sbatch
INFO: Workflow farnam (ID=852e6b72805bc1ab) is executed successfully with 1 completed step.


### FastGWA exome data

In [2]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_exome_data/f3393_hearing_aid_exomes
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f3393_exomes-fastgwa.sbatch
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/exome_files_snpsonly/ukb23155.filtered.merged.bed
genoFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
sampleFile=
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_aid_f3393_128254ind_exomes
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_aid_f3393_128254ind_exomes
formatFile_fastgwa=~/project/UKBB_GWAS_dev/data/fastGWA_template.yml
phenoCol=hearing_aid_cat
covarCol=sex
qCovarCol=age_final_aid
bgenMinMAF=0.001
grmFile=

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --genoFile $genoFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2021-01-11_f3393_exomes-fastgwa.sbatch
INFO: Workflow farnam (ID=3f949028f44f4152) is executed successfully with 1 completed step.



### Regenie exome data

In [4]:
tpl_file=../farnam.yml
lmm_dir_regenie=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/REGENIE_results/results_exome_data/f3393_hearing_aid_exomes
lmm_sbatch_regenie=../output/$(date +"%Y-%m-%d")_f3393_hearing_aid_exome-regenie.sbatch
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
#Use original bed files from the UKBB
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/exome_files_snpsonly/ukb23155.filtered.merged.bed
# In this case the bed files for exome data
genoFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
# No sample file is used
sampleFile=
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_aid_f3393_128254ind_exomes
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_aid_f3393_128254ind_exomes
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
phenoCol=hearing_aid_cat
covarCol=sex
qCovarCol=age_final_aid
bsize=1000
lowmem_prefix=f3393_regenie
trait=bt
bgenMinMAF=0.001
maf_filter=0.01
geno_filter=0.01
hwe_filter=0
mind_filter=0.02
minMAC=4
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --sampleFile $sampleFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_prefix $lowmem_prefix
    --trait $trait 
    --bgenMinMAF $bgenMinMAF 
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2021-01-12_f3393_hearing_aid_exome-regenie.sbatch
INFO: Workflow farnam (ID=9e2a5a5c798d943c) is executed successfully with 1 completed step.



### Bolt-LMM job

In [2]:
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/f3393_hearing_aid
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_f3393_hearing_aid_imp-bolt.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_Hearing_aid_f3393
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_Hearing_aid_f3393
phenoCol=hearing_aid_cat
covarCol=sex
qCovarCol=age_final_aid

lmm_args="""boltlmm
    --cwd $lmm_dir_bolt 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_bolt 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp    
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_bolt \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-20_f3393_hearing_aid_imp-bolt.sbatch
INFO: Workflow farnam (ID=c482417dfcba9f23) is executed successfully with 1 completed step.



### LD clumping job

In [ ]:
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/f3393_hearing_aid
clumping_sos=~/project/bioworkflows/GWAS/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_f3393_hearing_aid_ldclumping.sbatch
sumstatsFiles=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f3393_hearing_aid/200828_UKBB_Hearing_aid_f3393_hearing_aid_cat.fastGWA.snp_stats.gz

clumping_args="""default 
    --cwd $clumping_dir 
    --bfile $bfile
    --bfile_ref $bfile_ref 
    --bgenFile $bgenFile
    --sampleFile $sampleFile 
    --sumstatsFiles $sumstatsFiles 
    --unrelated_samples $unrelated_samples 
    --ld_sample_size $ld_sample_size 
    --clump_field $clump_field
    --clump_p1 $clump_p1 
    --clump_p2 $clump_p2 
    --clump_r2 $clump_r2 
    --clump_kb $clump_kb 
    --clump_annotate $clump_annotate 
    --numThreads $numThreads 
    --job_size $clump_job_size
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $clumping_sos \
    --to-script $clumping_sbatch \
    --args "$clumping_args"

### Region extraction

In [4]:
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_Hearing_aid_f3393
extract_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/f3393_hearing_aid
extract_sos=~/project/UKBB_GWAS_dev/workflow/Region_Extraction_4.ipynb
extract_sbatch=../output/$(date +"%Y-%m-%d")_f3393_hearing_aid_imp-region.sbatch
region_file=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/f3393_hearing_aid/*.clumped_region
geno_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/UKBB_bgenfilepath.txt
sumstats_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f3393_hearing_aid/*.snp_stats.gz

extract_args="""default
    --cwd $extract_dir
    --region-file $region_file
    --pheno-path $phenoFile
    --geno-path $geno_path
    --bgen-sample-path $sampleFile
    --sumstats-path $sumstats_path
    --format-config-path $formatFile_fastgwa
    --unrelated-samples $unrelated_samples
    --job-size $extract_job_size
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $extract_sos \
    --to-script $extract_sbatch \
    --args "$extract_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-28_f3393_hearing_aid_imp-region.sbatch
INFO: Workflow farnam (ID=55af58288aa5cc61) is executed successfully with 1 completed step.



## Fine mapping

In [ ]:
finemap_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/fine_mapping/f3393_hearing_aid
finemap_sos=~/project/UKBB_GWAS_dev/workflow/SuSiE_test.ipynb
finemap_sbatch=../output/$(date +"%Y-%m-%d")_f3393_hearing_aid_imp-finemap.sbatch
region_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/f3393_hearing_aid
region_file=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/f3393_hearing_aid/200828_UKBB_Hearing_aid_f3393_hearing_aid_cat.fastGWA.snp_stats.clumped_region
sumstats_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f3393_hearing_aid/*.snp_stats.gz
N=230411
container_lmm=/home/dc2325/scratch60/lmm_v_1_4.sif

finemap_args="""default
    --cwd $finemap_dir
    --region_dir $region_dir
    --region_file $phenoFile
    --sumstats_path $sumstats_path
    --container_lmm
"""
sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $finemap_sos \
    --to-script $finemap_sbatch \
    --args "$finemap_args"

## 2. Hearing difficulty/problems f.2247

### FastGWA job white British

In [ ]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2247_hearing_difficulty
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f2247_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_Hearing_difficulty_f2247
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_Hearing_difficulty_f2247
phenoCol=hearing_diff_new
covarCol=sex
qCovarCol=age_final_diff

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

### FastGWA all white

In [3]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2247_hearing_difficulty_expandedwhite
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f2247_expanded_white_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/120120_UKBB_Hearing_difficulty_f2247_expandedwhite
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/120120_UKBB_Hearing_difficulty_f2247_expandedwhite
phenoCol=hearing_diff_new
covarCol=sex
qCovarCol=age_final_diff

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-12-02_f2247_expanded_white_imp-fastgwa.sbatch
INFO: Workflow farnam (ID=9da5aed9605f3f9f) is executed successfully with 1 completed step.


### FastGWA exome data

In [2]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_exome_data/f2247_hearing_difficulty_exomes
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f2247_exomes-fastgwa.sbatch
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/exome_files_snpsonly/ukb23155.filtered.merged.bed
genoFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
sampleFile=
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_difficulty_f2247_171970ind_exomes
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_difficulty_f2247_171970ind_exomes
formatFile_fastgwa=~/project/UKBB_GWAS_dev/data/fastGWA_template.yml
phenoCol="hearing_diff_new"
covarCol=sex
qCovarCol=age_final_diff
bgenMinMAF=0.001

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --genoFile $genoFile 
    --phenoFile $phenoFile
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2021-01-12_f2247_exomes-fastgwa.sbatch
INFO: Workflow farnam (ID=372a90dd555a65fb) is executed successfully with 1 completed step.



### Regenie exome data

In [1]:
tpl_file=../farnam.yml
lmm_dir_regenie=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/REGENIE_results/results_exome_data/f2247_hearing_difficulty_exomes
lmm_sbatch_regenie=../output/$(date +"%Y-%m-%d")_f2247_hearing_difficulty_exome-regenie.sbatch
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
#Use original bed files from the UKBB
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/exome_files_snpsonly/ukb23155.filtered.merged.bed
# In this case the bed files for exome data
genoFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
# No sample file is used
sampleFile=
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_difficulty_f2247_171970ind_exomes
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_difficulty_f2247_171970ind_exomes
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
phenoCol=hearing_diff_new
covarCol=sex
qCovarCol=age_final_diff
bsize=1000
lowmem_prefix=f2247_regenie
trait=bt
bgenMinMAF=0.001
maf_filter=0.01
geno_filter=0.01
hwe_filter=0
mind_filter=0.02
minMAC=4
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --sampleFile $sampleFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_prefix $lowmem_prefix
    --trait $trait 
    --bgenMinMAF $bgenMinMAF 
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2021-01-15_f2247_hearing_difficulty_exome-regenie.sbatch
INFO: Workflow farnam (ID=w3e2c65fdd9b5b54f) is executed successfully with 1 completed step.



### LD clumping job

In [ ]:
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/f2247_hearing_difficulty
clumping_sos=~/project/bioworkflows/GWAS/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_f2247_hearing_difficulty_ldclumping.sbatch
sumstatsFiles=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2247_hearing_difficulty/200828_UKBB_Hearing_difficulty_f2247_hearing_diff_new.fastGWA.snp_stats.gz

clumping_args="""default 
    --cwd $clumping_dir 
    --bfile $bfile
    --bfile_ref $bfile_ref 
    --bgenFile $bgenFile
    --sampleFile $sampleFile 
    --sumstatsFiles $sumstatsFiles 
    --unrelated_samples $unrelated_samples 
    --ld_sample_size $ld_sample_size 
    --clump_field $clump_field
    --clump_p1 $clump_p1 
    --clump_p2 $clump_p2 
    --clump_r2 $clump_r2 
    --clump_kb $clump_kb 
    --clump_annotate $clump_annotate 
    --numThreads $numThreads 
    --job_size $clump_job_size
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $clumping_sos \
    --to-script $clumping_sbatch \
    --args "$clumping_args"

## 3. Hearing difficulty with background noise f.2257

### FastGWA job white British

In [ ]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2257_hearing_background_noise
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f2257_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_Hearing_background_noise_f2257
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_Hearing_background_noise_f2257
phenoCol=hearing_noise_cat
covarCol=sex
qCovarCol=age_final_noise

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

### FastGWA job all white

In [6]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2257_hearing_background_noise_expandedwhite
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f2257_expandedwhite_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/120120_UKBB_Hearing_background_noise_f2257_expandedwhite
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/120120_UKBB_Hearing_background_noise_f2257_expandedwhite
phenoCol=hearing_noise_cat
covarCol=sex
qCovarCol=age_final_noise

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb dewan \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running dewan: Configuration for Yale `pi_dewan` partition cluster
INFO: dewan is completed.
INFO: dewan output:   ../output/2020-12-02_f2257_expandedwhite_imp-fastgwa.sbatch
INFO: Workflow dewan (ID=5e423268c0ec2f3b) is executed successfully with 1 completed step.


### Regenie exome data

In [2]:
tpl_file=../farnam.yml
lmm_dir_regenie=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/REGENIE_results/results_exome_data/f2257_hearing_noise_exomes
lmm_sbatch_regenie=../output/$(date +"%Y-%m-%d")_f2257_hearing_noise_exome-regenie.sbatch
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
#Use original bed files from the UKBB
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/exome_files_snpsonly/ukb23155.filtered.merged.bed
# In this case the bed files for exome data
genoFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
# No sample file is used
sampleFile=
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_background_noise_f2257_175531ind_exomes
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_Hearing_background_noise_f2257_175531ind_exomes
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
phenoCol=hearing_noise_cat
covarCol=sex
qCovarCol=age_final_noise
bsize=1000
lowmem_prefix=f2257_regenie
trait=bt
bgenMinMAF=0.001
maf_filter=0.01
geno_filter=0.01
hwe_filter=0
mind_filter=0.02
minMAC=4
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --sampleFile $sampleFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_prefix $lowmem_prefix
    --trait $trait 
    --bgenMinMAF $bgenMinMAF 
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2021-01-15_f2257_hearing_noise_exome-regenie.sbatch
INFO: Workflow farnam (ID=w5684e525fc596679) is executed successfully with 1 completed step.



### LD clumping job

In [ ]:
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/f2257_hearing_background_noise
clumping_sos=~/project/bioworkflows/GWAS/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_f2257_hearing_background_noise_ldclumping.sbatch
sumstatsFiles=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2257_hearing_background_noise/200828_UKBB_Hearing_background_noise_f2257_hearing_noise_cat.fastGWA.snp_stats.gz

clumping_args="""default 
    --cwd $clumping_dir 
    --bfile $bfile
    --bfile_ref $bfile_ref 
    --bgenFile $bgenFile
    --sampleFile $sampleFile 
    --sumstatsFiles $sumstatsFiles 
    --unrelated_samples $unrelated_samples 
    --ld_sample_size $ld_sample_size 
    --clump_field $clump_field
    --clump_p1 $clump_p1 
    --clump_p2 $clump_p2 
    --clump_r2 $clump_r2 
    --clump_kb $clump_kb 
    --clump_annotate $clump_annotate 
    --numThreads $numThreads 
    --job_size $clump_job_size
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $clumping_sos \
    --to-script $clumping_sbatch \
    --args "$clumping_args"

## 4. Combined phenotype f.2247 & f.2257

### FastGWA job white British

In [ ]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2247_f2257_combined
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f2247_f2257_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_f2247_f2257
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_f2247_f2257
phenoCol=f2247_f2257
covarCol=sex
qCovarCol=age

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

### FastGWA job all white

In [7]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2247_f2257_combined_expandedwhite
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_f2247_f2257_expandedwhite_imp-fastgwa.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/120120_UKBB_f2247_f2257_expandedwhite
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/120120_UKBB_f2247_f2257_expandedwhite
phenoCol=f2247_f2257
covarCol=sex
qCovarCol=age

lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb dewan \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running dewan: Configuration for Yale `pi_dewan` partition cluster
INFO: dewan is completed.
INFO: dewan output:   ../output/2020-12-02_f2247_f2257_expandedwhite_imp-fastgwa.sbatch
INFO: Workflow dewan (ID=cca88c31ebd37730) is executed successfully with 1 completed step.


### Bolt-LMM job

In [3]:
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/f2247_f2257_combined
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_f2247_f2257_imp-bolt.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_f2247_f2257
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200828_UKBB_f2247_f2257
phenoCol=f2247_f2257
covarCol=sex
qCovarCol=age
lmm_option='lmmForceNonInf'

lmm_args="""boltlmm
    --cwd $lmm_dir_bolt 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_bolt 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --lmm_option $lmm_option
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp    
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_bolt \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-28_f2247_f2257_imp-bolt.sbatch
INFO: Workflow farnam (ID=6f1d2712738ba187) is executed successfully with 1 completed step.



### Regenie exome data

In [4]:
tpl_file=../farnam.yml
lmm_dir_regenie=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/REGENIE_results/results_exome_data/f2247_f2257_combined_exomes
lmm_sbatch_regenie=../output/$(date +"%Y-%m-%d")_f2247_f2257_combined_exome-regenie.sbatch
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
#Use original bed files from the UKBB
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/exome_files_snpsonly/ukb23155.filtered.merged.bed
# In this case the bed files for exome data
genoFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`
# No sample file is used
sampleFile=
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_f2247_f2257_136862ind_exomes
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/phenotypes_exome_data/010421_UKBB_f2247_f2257_136862ind_exomes
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
phenoCol=f2247_f2257
covarCol=sex
qCovarCol=age_combined
bsize=1000
lowmem_prefix=f2257_regenie
trait=bt
bgenMinMAF=0.001
maf_filter=0.01
geno_filter=0.01
hwe_filter=0
mind_filter=0.02
minMAC=4
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=20
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --sampleFile $sampleFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_prefix $lowmem_prefix
    --trait $trait 
    --bgenMinMAF $bgenMinMAF 
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2021-01-15_f2247_f2257_combined_exome-regenie.sbatch
INFO: Workflow farnam (ID=we7c1c22aeceb6f8d) is executed successfully with 1 completed step.



### LD clumping job

In [ ]:
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/f2247_f2257_combined
clumping_sos=~/project/bioworkflows/GWAS/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_f2247_f2257_combined_ldclumping.sbatch
sumstatsFiles=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2247_f2257_combined/200828_UKBB_f2247_f2257_f2247_f2257.fastGWA.snp_stats.gz

clumping_args="""default 
    --cwd $clumping_dir 
    --bfile $bfile
    --bfile_ref $bfile_ref 
    --bgenFile $bgenFile
    --sampleFile $sampleFile 
    --sumstatsFiles $sumstatsFiles 
    --unrelated_samples $unrelated_samples 
    --ld_sample_size $ld_sample_size 
    --clump_field $clump_field
    --clump_p1 $clump_p1 
    --clump_p2 $clump_p2 
    --clump_r2 $clump_r2 
    --clump_kb $clump_kb 
    --clump_annotate $clump_annotate 
    --numThreads $numThreads 
    --job_size $clump_job_size
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $clumping_sos \
    --to-script $clumping_sbatch \
    --args "$clumping_args"

## 5. Hudson plot

### f2247_f2257 combined vs Hdiff wells paper

In [2]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots/hearing_impairment/
hudson_sbatch=../output/$(date +"%Y-%m-%d")_f2247_f2257_vs_hdiff_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f2247_f2257_combined/200828_UKBB_f2247_f2257_f2247_f2257.fastGWA.snp_stats.gz
sumstats_2=/home/dc2325/project/HI_UKBB/2019_Wells_sumstats/HD_EA_gwas_sumstats.txt.gz
toptitle="f2247_f2257_combined"
bottomtitle="Hdiff_Wells_GWAS"
highlight_p_top=0.0
highlight_p_bottom=0.0
pval_filter=5e-08
highlight_snp=/home/dc2325/project/HI_UKBB/2019_Wells_sumstats/hdiff_snps_wells
job_size=1
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
    --highlight_snp $highlight_snp
    --container_lmm $container_lmm
"""
sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-26_f2247_f2257_vs_hdiff_hudson.sbatch
INFO: Workflow farnam (ID=f34e7c33d379875d) is executed successfully with 1 completed step.



## f3393 vs haid wells paper

In [2]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots/hearing_impairment/
hudson_sbatch=../output/$(date +"%Y-%m-%d")_f3393_vs_Haid_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/f3393_hearing_aid/200828_UKBB_Hearing_aid_f3393_hearing_aid_cat.fastGWA.snp_stats.gz
sumstats_2=/home/dc2325/project/HI_UKBB/2019_Wells_sumstats/HAID_EA_gwas_sumstats.txt.gz
toptitle="f_3393_hearing_aid"
bottomtitle="Haid_Wells_GWAS"
highlight_p_top=0.0
highlight_p_bottom=0.0
pval_filter=5e-08
highlight_snp=/home/dc2325/project/HI_UKBB/2019_Wells_sumstats/haid_snps_wells
job_size=1
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
    --highlight_snp $highlight_snp
    --container_lmm $container_lmm
"""
sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-22_f3393_vs_Haid_hudson.sbatch
INFO: Workflow farnam (ID=f48b56e3e091ef16) is executed successfully with 1 completed step.



## 6. Fine mapping

### f.3391 hearing aid

In [1]:
tpl_file=../farnam.yml
finemap_sos=~/project/UKBB_GWAS_dev/SuSiE_RSS.ipynb
finemap_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/fine_mapping/f3393_hearing_aid
finemap_sbatch=../output/$(date +"%Y-%m-%d")_f3393_hearing_aid_susie.sbatch
sumstatFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/f3393_hearing_aid/10_126783170_126813028/200828_UKBB_Hearing_aid_f3393_hearing_aid_cat.fastGWA.snp_stats_10_126783170_126813028.sumstats.gz
ldFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/f3393_hearing_aid/10_126783170_126813028/200828_UKBB_Hearing_aid_f3393_hearing_aid_cat.fastGWA.snp_stats_10_126783170_126813028.sample_ld.gz
N=230411
job_size=1
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif

finemap_args="""default
    --cwd $finemap_dir
    --sumstatFile $sumstatFile
    --ldFile $ldFile
    --N $N
    --job_size $job_size
    --container_lmm $container_lmm
"""
sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $finemap_sos \
    --to-script $finemap_sbatch \
    --args "$finemap_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-15_f3393_hearing_aid_susie.sbatch
INFO: Workflow farnam (ID=fdf18b3eb0fe563d) is executed successfully with 1 completed step.

